In [1]:
import numpy as np
import time
import threading as th

import magpyx
from magpyx.utils import ImageStream

import purepyindi
from purepyindi import INDIClient
client0 = purepyindi.INDIClient('localhost', 7624)
client0.start()

# from skimage.registration import phase_cross_correlation
v_bias = np.array([[50,50,50]]).T

L = 12*u.mm # distance between FSM piezo actuators
B = L * np.cos(30*u.degree) # baseline distance of three piezos

max_stroke = 12*u.um
max_voltage = 100
D_per_V = max_stroke/max_voltage

def set_tip_tilt(tip, tilt, dZ=0):
    tip = tip.to_value(u.radian)
    tilt = tilt.to_value(u.radian)

    dA = get_A(tip, dZ)
    dB = get_B(tip, tilt, dZ)
    dC = get_C(tip, tilt, dZ)
    print(f'Displacements: A = {dA:.2e}, {dB:.2e}, {dC:.2e}. ')

    dvA = (dA/D_per_V).decompose().value
    dvB = (dB/D_per_V).decompose().value
    dvC = (dC/D_per_V).decompose().value
    print(f'Delta Voltages: A = {dvA:.2f}, B = {dvB:.2f}, C = {dvC:.2f}. ')

    # self.voltages = self.voltages + np.array([dvA, dvB, dvC])
    
    return dvA, dvB, dvC

def get_A(alpha, Z):
    return (Z + 2./3. * B * alpha).to(u.m)

def get_B(alpha, beta, Z):
    return (0.5 * L * beta + Z - 1./3. * B * alpha).to(u.m)

def get_C(alpha, beta, Z):
    return (Z - 1./3. * B * alpha - 1./2. * L * beta).to(u.m)


ERROR:purepyindi:Connection refused: [Errno 111] Connection refused


ConnectionRefusedError: [Errno 111] Connection refused

In [58]:

fsm = ImageStream('fsm')

fsm.write(np.array([[0,0,0]]).transpose())

In [39]:
# time.sleep(2)
fsm.write(np.array([[0,0,0]]).T)

In [35]:
class Process(th.Timer):  
    def run(self):  
        while not self.finished.wait(self.interval):  
            self.function(*self.args, **self.kwargs)
# process = Process(0.1, print, ['Repeating']) 
# process.start()
# time.sleep(5)
# process.cancel()

In [63]:
volts = np.array([[[50,50,50]], [[50,50,70]]])
freq = 10
try:
    print('Modulating FSM ...')
    i = 0
    while True:
        fsm.write(volts[i%2].T)
        i += 1 
        time.sleep(1/freq)
except KeyboardInterrupt:
    print('FSM modulation stopped!')
    fsm.write(v_bias)


Modulating FSM ...
FSM modulation stopped!
